In [95]:
import numpy as np
import os,sys,inspect
import importlib
%pdb on

# import my functions:
# (1) find the path
dirpath = os.getcwd()
i = 0
while(os.path.basename(dirpath) != "IEMS450_HW") and i <10:
    dirpath = os.path.dirname(dirpath) 
    i = i + 1
targetdir = dirpath + '/HW2/functions'
if targetdir not in sys.path:
    sys.path.insert(0,targetdir)
    
# (2) import
import exercise_4_2 as obj
importlib.reload(obj)

Automatic pdb calling has been turned ON


<module 'exercise_4_2' from '/Users/jingyuanwang/GitHub/IEMS450_HW/HW2/functions/exercise_4_2.py'>

In [204]:
#x0 = np.array([0,0.5])
x0 = np.array([-0,-1])

In [205]:
f = obj.f()
m = obj.m(f, x=x0)

In [206]:
# steepest descent
d0 = -m.der_x0
d0 = d0/np.linalg.norm(d0) 

# newton
d_newton = np.linalg.solve(m.H_x0, -m.der_x0)
print('Newton step length {}'.format(np.linalg.norm(d_newton)) )

# steps for different delta: trust region
import scipy.optimize as opt
d_k = {}

d_start = np.array([0.2,0.2])
for i, Delta in enumerate( np.arange(0, 2, 0.2)[1:]):
    cons = ({'type': 'ineq', 'fun': lambda x:  -x[0]**2 - x[1]**2 + Delta**2 })
    res=opt.minimize(m.value_step, d_start, jac=m.der_1st, hess=m.der_2nd,
                     constraints=cons)
    d_k[i]= res.x
    
del d_start

Newton step length 1.001133144839459


/Users/jingyuanwang/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method SLSQP does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


In [207]:
# check whether hession is PD
np.linalg.eigvals(m.H_x0)

array([42., 20.])

In [202]:
# plot
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.font_manager import FontProperties

x_min = x0[0] - 2
x_max = x0[0] + 2
y_min = x0[1] - 2
y_max = x0[1] + 2

delta = 0.05
x = np.arange(x_min, x_max, delta)
y = np.arange(y_min, y_max, delta)
X, Y = np.meshgrid(x, y)

Z=f.value( (X,Y) )
Z_hat=m.value( X,Y )


fig = plt.figure(figsize=(8,8))

# 1. plot contour
cs = plt.contour(X, Y, Z_hat, levels=[-20,-10,1, 5,10,20,40, 60,100], cmap='autumn'
                , norm=colors.PowerNorm(gamma=1. / 2.))
cs_true = plt.contour(X, Y, Z, levels=[0.1, 1, 5,10,20,40, 60,100], cmap='bone', linestyles='dotted'
                     , norm=colors.PowerNorm(gamma=1. / 3.))
plt.scatter(x0[0], x0[1], color='black' )
#plt.text(x0[0]+0.1, x0[1]-0.1, s='x', fontsize=20 )
# labels
plt.clabel(cs, inline=1, fontsize=10, fmt='%3.1f');
plt.clabel(cs_true, inline=1, fontsize=10, fmt='%3.1f');
cs.collections[1].set_label('m')
cs_true.collections[1].set_label('true function')

# 2. plot newton and steepest descent， trust region
for i,d in d_k.items():
    a = plt.arrow( x0[0], x0[1], 
           d[0], d[1], color='c', length_includes_head=True,
          head_width=0.05, head_length=0.1)

d  = np.squeeze(d0)
sd = plt.arrow( x0[0], x0[1], 
           d[0], d[1], color='lightblue', length_includes_head=True,
          head_width=0.05, head_length=0.1)
d  = np.squeeze(d_newton)
nt = plt.arrow( x0[0], x0[1], 
           d[0], d[1], color='gray', length_includes_head=True,
          head_width=0.05, head_length=0.1)

# legend
fontP = FontProperties()
fontP.set_size('x-large')
plt.legend([nt,sd,a,cs.collections[1], cs_true.collections[1]], ['Newton','-g/|g|','trust region',
                                                                 'm','true value'],
           bbox_to_anchor=(1.05, 1), loc='upper left', prop=fontP);